In [11]:
import pandas as pd 
from smolagents import load_tool, CodeAgent, HfApiModel, DuckDuckGoSearchTool, OpenAIServerModel, LiteLLMModel, tool
from vllm import LLM
from pathlib import Path

In [12]:
issue_data = pd.read_parquet('../data/data.parquet')

row = issue_data.iloc[4]

In [13]:
row['problem_statement']

'KeyError: \'version_1_3_or_later\' when parsing certain VOTables\n### Description\n\nWhen parsing VOTables (for instance) VOTables with empty integer literals in some places (e.g., MIN value="" in a VALUES element), astropy crashes with\r\n\r\n```\r\nKeyError: \'version_1_3_or_later\'\r\n```\n\n### Expected behavior\n\nWell, you *could* argue that at least some such VOTables should be rejected (a NULL value in a MIN, indeed, does not make sense) with a sensible error message; with the patch proposed in the accompanying PR, astropy emits a warning.  I notice in passing that the reproducer table passes stilts votlint.  I also note in passing that against the warning, a value="" is accepted in a PARAM (as it should, as this is the way to express NULLs).\r\n\r\nMy fix in the accompanying PR just stops the crashing, leading to a workable table.\n\n### How to Reproduce\n\nTry\r\n\r\n```python\r\nfrom astropy import table\r\ntable.Table.read("with-empty-min.vot", format="votable")\r\n```\r\n

In [14]:
from typing import Union, Optional

def tree(directory: Union[str, Path], max_depth: Optional[int] = None, 
         _prefix: str = "", _depth: int = 0) -> list[str]:
    """
    Recursively list the contents of a directory in a tree-like format.
    
    Args:
        directory: The directory path to list
        max_depth: Maximum depth to traverse (None for unlimited)
        _prefix: Used internally for formatting the tree structure
        _depth: Used internally to track current recursion depth
        
    Returns:
        List of strings representing the tree structure
    """
    # Convert to Path object if string is provided
    directory = Path(directory)
    
    # Check if we've reached max depth
    if max_depth is not None and _depth > max_depth:
        return []
    
    # Get all items in the directory
    try:
        items = sorted(directory.iterdir(), key=lambda p: (p.is_file(), p.name.lower()))
    except PermissionError:
        return [f"{_prefix}├── {directory.name} [Permission Denied]"]
    except FileNotFoundError:
        return [f"{_prefix}Directory not found: {directory}"]
    
    # Initialize the result with the current directory
    if _depth == 0:
        result = [str(directory) + "/"]
    else:
        result = []
    
    # Process each item
    count = len(items)
    for i, item in enumerate(items):
        # Determine if this is the last item
        is_last = i == count - 1
        
        # Choose the appropriate connector based on whether this is the last item
        connector = "└── " if is_last else "├── "
        
        # Add the current item to the result
        result.append(f"{_prefix}{connector}{item.name}" + ("/" if item.is_dir() else ""))
        
        # If this is a directory, recursively process it
        if item.is_dir():
            # Extend prefix for children
            # Use spaces for last item's children, pipes for others
            extended_prefix = _prefix + ("    " if is_last else "│   ")
            
            # Recursively process the subdirectory
            subtree = tree(
                item, 
                max_depth=max_depth, 
                _prefix=extended_prefix,
                _depth=_depth + 1
            )
            
            # Add the subtree to the result
            result.extend(subtree)
    
    return result

def build_tools(instance_id):

    REPO_PATH = Path(f'../data/repos/repo__{instance_id}')

    @tool
    def get_file_content(file_path: str) -> str:
        """Get the content of a file in the code repository
        Args:
            file_path: The relative path to the file to get the content of
        Returns:
            The content of the file
        """
        return Path(REPO_PATH / file_path).read_text()
    
    @tool
    def list_files(extension: str) -> list[Path]:
        """List all files in the code repository with a given extension
        Args:
            extension: The extension of the files to list, e.g. "py" , "md", "json"
        Returns:
            A list of file paths
        """
        return [str(file.relative_to(REPO_PATH)) for file in REPO_PATH.glob(f'**/*.{extension}')]
    
    @tool
    def print_tree(directory: str, max_depth: Optional[int] = 1) -> None:
        """
        Print directory contents in a tree-like format.
        
        Args:
            directory: The directory path to list
            max_depth: Maximum depth to traverse (None for unlimited)
        """
        tree_list = tree(REPO_PATH / directory, max_depth)
        for line in tree_list:
            print(line)
    
    return [get_file_content, list_files, print_tree]


In [15]:
import os 
from typing import Union, Optional, List
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))



@tool
def print_tree(directory: str, max_depth: Optional[int] = None) -> None:
    """
    Print directory of the repository contents in a tree-like format.
    
    Args:
        directory: The directory relative to the repository path to list
        max_depth: Maximum depth to traverse (None for unlimited)
    """
    tree_list = tree(directory, max_depth)
    for line in tree_list:
        print(line)
print_tree(f'../data/repos/repo__{row["instance_id"]}', 2)

../data/repos/repo__astropy__astropy-16830/
├── .circleci/
│   └── config.yml
├── .devcontainer/
│   ├── devcontainer.json
│   └── welcome-message.txt
├── .github/
│   ├── ISSUE_TEMPLATE/
│   │   ├── bug_report.yaml
│   │   ├── config.yml
│   │   └── feature_request.yaml
│   ├── workflows/
│   │   ├── CFF-test.yml
│   │   ├── check_changelog.yml
│   │   ├── check_milestone.yml
│   │   ├── ci_benchmark.yml
│   │   ├── ci_cron_daily.yml
│   │   ├── ci_cron_weekly.yml
│   │   ├── ci_workflows.yml
│   │   ├── codeql-analysis.yml
│   │   ├── open_actions.yml
│   │   ├── publish.yml
│   │   ├── stalebot.yml
│   │   ├── update_astropy_iers_data_pin.py
│   │   └── update_astropy_iers_data_pin.yml
│   ├── CODEOWNERS
│   ├── dependabot.yml
│   ├── labeler.yml
│   └── PULL_REQUEST_TEMPLATE.md
├── .pyinstaller/
│   ├── hooks/
│   │   ├── hook-astropy_iers_data.py
│   │   └── hook-skyfield.py
│   └── run_astropy_tests.py
├── astropy/
│   ├── _dev/
│   │   ├── __init__.py
│   │   └── scm_version.py


In [16]:
issue_data

,instance_id,repo,problem_statement,patch,test_patch,pull_number,base_commit,PASS_TO_PASS,FAIL_TO_PASS,issue_numbers
0,pylint-dev__astroid-2496,pylint-dev/astroid,TypeError: unsupported format string passed to...,diff --git a/ChangeLog b/ChangeLog\nindex 4560...,diff --git a/tests/test_inference.py b/tests/t...,2496,8d3cdbbe6685fd8cf211816bec56c90f38f1859e,[tests/test_inference.py::InferenceUtilsTest::...,[tests/test_inference.py::test_formatted_fstri...,[2492]
1,pylint-dev__astroid-2468,pylint-dev/astroid,Pylint checks against incorrect type with prop...,diff --git a/ChangeLog b/ChangeLog\nindex fdbb...,diff --git a/tests/test_inference.py b/tests/t...,2468,6db3a60553ff538a936d5dda23d67a3924a57f45,[tests/test_inference.py::InferenceUtilsTest::...,[tests/test_inference.py::InferenceTest::test_...,[2467]
2,astropy__astropy-17048,astropy/astropy,QTable cannot take `dimensionless_unscaled` wh...,diff --git a/astropy/table/table.py b/astropy/...,diff --git a/astropy/table/tests/test_table.py...,17048,d60f6b72cd525262bfd179331d9fe4474177918f,[astropy/table/tests/test_table.py::TestSetTab...,[astropy/table/tests/test_table.py::test_qtabl...,[17047]
3,astropy__astropy-16898,astropy/astropy,BUG: tables do not deal well with zero-sized s...,diff --git a/astropy/io/registry/core.py b/ast...,diff --git a/astropy/io/fits/tests/test_connec...,16898,ee6d087baf301c1d08db92e6e5b6d909d57e6fac,[astropy/io/fits/tests/test_connect.py::TestSi...,[astropy/io/fits/tests/test_connect.py::test_z...,[16897]
4,astropy__astropy-16830,astropy/astropy,KeyError: 'version_1_3_or_later' when parsing ...,diff --git a/astropy/io/votable/tree.py b/astr...,diff --git a/astropy/io/votable/tests/test_tre...,16830,e39f486fec48d87aa3677326167954370d7a7bf9,[astropy/io/votable/tests/test_tree.py::test_c...,[astropy/io/votable/tests/test_tree.py::test_v...,"[16825, 16826]"
5,astropy__astropy-16812,astropy/astropy,Provide a way to make a copy of a model with d...,diff --git a/astropy/modeling/core.py b/astrop...,diff --git a/astropy/modeling/tests/test_core....,16812,c241103c11954d3c1cfe3c1840b1ece72479c522,[astropy/modeling/tests/test_core.py::test_Mod...,[astropy/modeling/tests/test_core.py::test_res...,[16593]


In [17]:
llm_model_pth = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
model = LiteLLMModel(model_id=f"hosted_vllm/{llm_model_pth}", api_base='http://localhost:8000/v1')
agent = CodeAgent(
    tools=build_tools(row["instance_id"]),
    model=model,
)

In [18]:
agent = CodeAgent(
    tools=build_tools(row["instance_id"]),
    model=model,
)

In [19]:
agent.run(f"Tell me is going on in the setup file in this repo")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me is going on in the setup file in this repo                                                              │
│                                                                                                                 │
╰─ LiteLLMModel - hosted_vllm/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B ────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # List all .py files in the repository                                                                           
  all_py_files = list_files(file_path={'.py'})                                                                     
  print(f"The repository contains {len(all_py_files)} .py files: {all_py_files}")                                  
                                                                                                                   
  # Now, let's inspect each .py file to gather setup information                                                   
  setup_info = []                                                                                                  
  for file in all_py_files:                                                                                        
      print(f"\nProcessing {file}:")                                                                               
      try:                                                                                                         
          # Try to load the __init__.py file to get version information                                            
          with open(file, 'r') as f:                                                                               
              content = f.read()                                                                                   
              # Look for version variable                                                                          
              version_match = re.search(r"__version__ = .*?", content, re.MULTILINE)                               
              if version_match:                                                                                    
                  version = version_match.group(0).split()[-1].strip('"')                                          
                  setup_info.append(f"Version: {version}\n")                                                       
              # Look for comments indicating version                                                               
              comment_match = re.search(r'# Version .*?\n', content)                                               
              if comment_match:                                                                                    
                  setup_info.append(f"Version comment found:\n{content[comment_match.start():]}\n")                
              # Look for imports of libraries                                                                      
              import_match = re.search(r"import .*?\n", content)                                                   
              if import_match:                                                                                     
                  setup_info.append(f"Libraries imported (first few):")                                            
                  setup_info.append(f"{content[import_match.start():import_match.end()]}")                         
              # Look for __version__                                                                               
              version_line = re.search(r'(?i)^version .*?', content, re.MULTILINE)                                 
              if version_line:                                                                                     
                  version = version_line.group().strip().upper()                                                   
                  [38;2;2

Code parsing failed on line 38 due to: SyntaxError
The repository contains 3 .py files: ['document_qa.py', 'image_generator.py', 'translator.py']
     ^
Error: invalid syntax (<unknown>, line 38)

[Step 0: Duration 14.27 seconds| Input tokens: 2,206 | Output tokens: 1,048]

...

[Step 5: Duration 1.35 seconds| Input tokens: 20,821 | Output tokens: 8,438]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



Reached max steps.

[Step 6: Duration 1.35 seconds| Input tokens: 24,544 | Output tokens: 9,916]

'Error in generating final LLM output:\nlitellm.APIError: APIError: Hosted_vllmException - Connection error.'

In [20]:
df = pd.read_parquet("../data/data.parquet")

In [21]:
print(df.loc[0, 'problem_statement'])

TypeError: unsupported format string passed to NoneType.__format__
Regression in #2459

### Steps to reproduce
a.py:
```py
class A:
    def __init__(self):
        self._magnitude = None

    def name(self) -> str | None:
        if self._magnitude:
            return f"M {self._magnitude:.1f}"
```
```
pylint a.py
```
### Current behavior
```
  File "/Users/jwalls/release/lib/python3.12/site-packages/astroid/nodes/node_classes.py", line 4778, in _infer_from_values
    yield from nodes[0]._infer(context, **kwargs)
  File "/Users/jwalls/release/lib/python3.12/site-packages/astroid/nodes/node_classes.py", line 4695, in _infer
    formatted = format(value.value, format_spec.value)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: unsupported format string passed to NoneType.__format__
```



In [22]:
print(df.loc[0, 'patch'])

diff --git a/ChangeLog b/ChangeLog
index 4560e5d2b7..c08b1cbf2c 100644
--- a/ChangeLog
+++ b/ChangeLog
@@ -13,6 +13,9 @@ What's New in astroid 3.3.1?
 Release date: TBA
 
+* Fix a crash introduced in 3.3.0 involving invalid format strings.
+
+  Closes #2492
 
 
 What's New in astroid 3.3.0?
diff --git a/astroid/nodes/node_classes.py b/astroid/nodes/node_classes.py
index c1c7af36da..1924c78eba 100644
--- a/astroid/nodes/node_classes.py
+++ b/astroid/nodes/node_classes.py
@@ -4687,19 +4687,24 @@ def _infer(
                     uninferable_already_generated = True
                 continue
             for value in self.value.infer(context, **kwargs):
-                if not isinstance(value, Const):
-                    if not uninferable_already_generated:
-                        yield util.Uninferable
-                        uninferable_already_generated = True
-                    continue
-                formatted = format(value.value, format_spec.value)
-                yield Co